# CNN

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
img_size = (128, 128)
batch= 64

### Data Preprocessing

*Training Set*

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10
)

In [5]:
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=img_size, # try with 150, 150 later
    batch_size=batch,
    class_mode='binary',
    seed=11
)

Found 8000 images belonging to 2 classes.


*Test Set*

In [6]:
test_datagen = ImageDataGenerator(
    rescale=1.0/255
)

In [7]:
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=img_size, # try with 150, 150 later
    batch_size=batch,
    class_mode='binary',
    seed=11
)

Found 2000 images belonging to 2 classes.


### Building the CNN

*Initializing*

In [8]:
cnn = tf.keras.models.Sequential()

*Convolution*

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[img_size[0], img_size[1], 3]))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

*Pooling*

In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

In [11]:
#cnn.add(tf.keras.layers.Dropout(0.2, seed=11))

*Adding a second convolutional Layer*

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))
#cnn.add(tf.keras.layers.Dropout(0.2, seed=11))

*Flattening*

In [13]:
cnn.add(tf.keras.layers.Flatten())

*Full Connection*

In [14]:
cnn.add(tf.keras.layers.Dense(units=512, activation='relu'))
#cnn.add(tf.keras.layers.Dropout(0.25, seed=11))
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
#cnn.add(tf.keras.layers.Dropout(0.25, seed=11)) #Later on try with 2-3 Layers including dropout

Output Layer

In [15]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Training the CNN

*Compiling the CNN*

In [16]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 53824)             0

Training the CNN on Training set and evaluating it on the Test Set

In [17]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)

In [18]:
cnn.fit(x=training_set, validation_data=test_set, epochs=1000, callbacks=[early_stop], workers=8, use_multiprocessing=True)

Train for 125 steps, validate for 32 steps
Epoch 1/1000
125/125 [==============================] - 65s 523ms/step - loss: 0.7044 - accuracy: 0.5332 - val_loss: 0.6741 - val_accuracy: 0.6280
Epoch 2/1000
125/125 [==============================] - 62s 494ms/step - loss: 0.6732 - accuracy: 0.5932 - val_loss: 0.6745 - val_accuracy: 0.6150
Epoch 3/1000
125/125 [==============================] - 62s 494ms/step - loss: 0.6411 - accuracy: 0.6388 - val_loss: 0.6107 - val_accuracy: 0.6820
Epoch 4/1000
125/125 [==============================] - 63s 500ms/step - loss: 0.6106 - accuracy: 0.6655 - val_loss: 0.5957 - val_accuracy: 0.6920
Epoch 5/1000
125/125 [==============================] - 61s 487ms/step - loss: 0.5948 - accuracy: 0.6779 - val_loss: 0.5773 - val_accuracy: 0.6985
Epoch 6/1000
125/125 [==============================] - 58s 462ms/step - loss: 0.5698 - accuracy: 0.7063 - val_loss: 0.5685 - val_accuracy: 0.7135
Epoch 7/1000
125/125 [==============================] - 60s 478ms/step - lo

*Making Single Prediction*

In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [20]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [24]:
test_img = image.load_img(path='dataset/single_prediction/cat_or_dog_2.jpg', target_size=img_size)
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img, axis=0)
res = cnn.predict(test_img)
if res[0][0] == 1:
    pred = 'dog'
else:
    pred = 'cat'

In [25]:
print(pred)

cat


In [26]:
res

array([[0.]], dtype=float32)

Best Result till now:

- Image size: 128, 128
- Batch Size: 64
- NN Layers: (Conv(32, 3, relu) * 2 + MaxPool(2, 2)) + (Conv(64, 3, relu) * 2 + MaxPool(2, 2)) + Flatten() + (Dense(512, relu)) + (Dense(64, relu)) + Dense(1, sigmoid)
- Early Stop: patience(10) + monitor(val_accuracy) + mode(max)
- Fit: wrokers(8)
- Total Epochs: 55 + best(45)

In [27]:
#cnn.save('cat_or_dog_128_64_512_64_8910.h5')